In [1]:
import pathlib
import pandas as pd
import shutil
import logging

from idrstream.idr_stream import IdrStream

In [2]:
data_to_process = pd.read_csv("example_files/data_to_process.tsv", sep="\t", index_col=0)
# remove A1 wells because of irregular illumination
data_to_process = data_to_process[data_to_process["Well"] != "A1"]
data_to_process = data_to_process.reset_index(drop=True)
data_to_process

,Plate,Well,Frames,Well Number,Control Type,Original Gene Target
0,LT0001_02,A4,50,4,positive control,ENSG00000149503
1,LT0001_02,A15,50,15,negative control,negative control
2,LT0001_02,B2,50,26,negative control,negative control
3,LT0001_02,C1,50,49,positive control,ENSG00000149503
4,LT0001_02,C4,50,52,positive control,ENSG00000149503
...,...,...,...,...,...,...
6738,LT0603_06,O13,50,349,positive control,KIF11
6739,LT0603_06,O16,50,352,negative control,negative control
6740,LT0603_06,P3,50,363,negative control,negative control
6741,LT0603_06,P21,50,381,positive control,COPB


In [3]:
idr_id = "idr0013"
tmp_dir = pathlib.Path("tmp2/")
final_data_dir = pathlib.Path("mitocheck_control_features2/")
try:
    shutil.rmtree(tmp_dir)
    # shutil.rmtree(final_data_dir)
    pass
except:
    print("No files to remove")

stream = IdrStream(idr_id, tmp_dir, final_data_dir, log='idr_stream2.log')

In [4]:
aspera_path = pathlib.Path("/home/roshankern/.aspera/ascli/sdk/ascp")
aspera_key_path = pathlib.Path("example_files/asperaweb_id_dsa.openssh")
screens_path = pathlib.Path("example_files/idr0013-screenA-plates.tsv")
save_dir = pathlib.Path("data/")

stream.init_downloader(aspera_path, aspera_key_path, screens_path)

In [5]:
fiji_path = pathlib.Path("/home/roshankern/Desktop/Github/IDR_stream_prototype/preprocess_data/Fiji.app")
stream.init_preprocessor(fiji_path)

In [6]:
nuclei_model_specs = {
            "model_type": "cyto",
            "channels": [0, 0],
            "diameter": 0,
            "flow_threshold": 0.8,
            "cellprob_threshold": 0,
            "remove_edge_masks": True,
        }
stream.init_segmentor(nuclei_model_specs)

In [7]:
config_path = pathlib.Path("example_files/DP_files/mitocheck_profiling_config.json")
checkpoint_path = pathlib.Path("example_files/DP_files/efficientnet-b0_weights_tf_dim_ordering_tf_kernels_autoaugment.h5")
stream.copy_DP_files(config_path, checkpoint_path)

In [8]:
stream.run_stream(data_to_process, batch_size=3, start_batch=0, batch_nums=[0], extra_metadata=["object_outlines"])

0      [[668   7]\n [667   8]\n [666   8]\n [665   9]...
1      [[1011    7]\n [1010    8]\n [1009    8]\n [10...
2      [[895   8]\n [894   9]\n [893   9]\n [892   9]...
3      [[528   9]\n [527  10]\n [526  10]\n [525  10]...
4      [[478  10]\n [477  11]\n [476  11]\n [475  12]...
                             ...                        
280    [[ 580  982]\n [ 579  983]\n [ 578  984]\n [ 5...
281    [[ 196  984]\n [ 195  985]\n [ 194  985]\n [ 1...
282    [[ 561  984]\n [ 560  985]\n [ 559  985]\n [ 5...
283    [[1310  989]\n [1309  990]\n [1308  990]\n [13...
284    [[ 199  999]\n [ 198 1000]\n [ 197 1001]\n [ 1...
Name: object_outline, Length: 285, dtype: object
(285, 1291)
0      [[418   4]\n [417   5]\n [416   5]\n [415   5]...
1      [[236  12]\n [235  13]\n [234  13]\n [233  13]...
2      [[471  14]\n [470  15]\n [469  15]\n [468  15]...
3      [[643  16]\n [642  17]\n [641  18]\n [640  19]...
4      [[1159   18]\n [1158   19]\n [1157   19]\n [11...
                           